In [8]:
# In this notebook we will generate the text using the learned model

In [1]:
# import necessary packages
from keras.models import Sequential
import numpy
import sys
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [1]:
# We need to load the ASCII text for the poetry into memory and convert all of the characters 
# to lowercase to reduce the vocabulary that the network must learn.

In [2]:
# load ascii text and covert to lowercase
filename = "poetry_WH_Auden.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [2]:
# Reverse mapping that we can use to convert the integers back to characters 
# so that we can understand the predictions.

In [3]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [4]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  2525
Total Vocab:  33


In [3]:
# Now we need to actually make predictions.First we will start off with a seed sequence as input, generate the next 
# character then update the seed sequence to add the generated character on the end and trim off the 
# first character. This process is repeated for as long as we want to predict new characters

In [23]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  2425


In [4]:
# We must transform the list of input sequences into the form 
# [samples, time steps, features] expected by an LSTM network.
# Next we need to rescale the integers to the range 0-to-1 to make the patterns easier to 
# learn by the LSTM network that uses the sigmoid activation function by default.

In [7]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [8]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [5]:
# load the weights of the learned model.

In [10]:
# load the network weights
filename = "weights-improvement-96-0.0192.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [6]:
# generate a random seen

In [36]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" d among a hundred cities
and wholly given over to unfamiliar affections,
to find his happiness in an "


In [7]:
# generate the characters

In [43]:
# generate characters
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

re fairly accustomed,
and each in the cell of himself is almost convinced of his freedom,
a few thousand will think of this day
as one thinks of a day when one did something slightly unusual.

what instruments we have agree
the day of his death was a dark cold day.

far from his illness
the wolves ran on through the evergreen forests,
the peasant river was untempted by the fashionable quays;
by mourning tongues
the death of the poet was kept from his poems.

but for him it was his last afternoon
Done.
